In [16]:
import pandas as pd
from keras.layers import Dense, Dropout, Activation, Input
from keras.models import Model
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import datetime
import json
import numpy as np

from utils import *

In [17]:
baseFile = '../DataExplore/eegPatternBandDHT.csv'
base = pd.read_csv(baseFile)
base = base.drop_duplicates(keep='first')

In [18]:
print(base)

    kurtosis      mean  meanAmplitude  meanFrequency    median  \
0    11.5723 -0.310300        0.64558       0.084093 -0.615560   
1     7.0162 -0.314940        0.20093       0.051268 -0.827660   
2     3.6025 -1.321800        1.23030       0.036277 -2.901600   
4     3.9980  1.023900        5.20380       0.045542 -0.903780   
5     7.7734 -2.064500        1.09900       0.045057 -2.650700   
..       ...       ...            ...            ...       ...   
72    3.2226 -0.063058        1.74540       0.045462 -0.007136   
73    3.9027 -1.159700        1.05180       0.042820 -2.256000   
74    2.7070 -0.543080        2.31060       0.055341 -1.003900   
75    4.7007 -0.427010        2.99220       0.045443  0.125820   
76    3.9027 -1.159700        1.05180       0.042820 -2.256000   

    medianFrequency  skewness  totalPower  variance zzclassification  ...  \
0          0.031330  1.245500   2494.0973   60.8402               TA  ...   
1          0.024136  1.058400   2783.4228   67.9031  

In [19]:
features = base.drop(columns=['zzclassification'])
classes = base.loc[:, ["zzclassification"]].values
featureList = "kurtosis,mean,meanAmplitude,meanFrequency,median,medianFrequency,skewness,totalPower,variance,zzclassification,meandelta,meanAmplitudedelta,powerdelta,meanAdelta,meanFidelta,meanalpha,meanAmplitudealpha,poweralpha,meanAalpha,meanFialpha,meantheta,meanAmplitudetheta,powertheta,meanAtheta,meanFitheta,meanbeta,meanAmplitudebeta,powerbeta,meanAbeta,meanFibeta,meangamma,meanAmplitudegamma,powergamma,meanAgamma,meanFigamma".split(",")

exclusionList = "zzclassification,meangamma,meanAmplitudegamma,meanAgamma,meanFigamma,medianFrequency,meanFrequency,meanAbeta,meanFibeta,meanAalpha,meanFialpha,meanAtheta,meanFitheta,meanAdelta,meanFidelta"
for i in exclusionList.split(","):
    if i in featureList:
        featureList.remove(i)

features = selectFeature(features=features, featureList=featureList)
[
            "kurtosis",
            "mean",
            "meanAmplitude",
            "meanFrequency",
            "median",
            "medianFrequency",
            "skewness",
            "totalPower",
            "variance",
            "meanAmplitudedelta",
            "powerdelta",
            "meanAdelta",
            "meanFidelta",
            "meanAmplitudealpha",
            "poweralpha",
            "meanAalpha",
            "meanFialpha",
            "meanAmplitudetheta",
            "powertheta",
            "meanAtheta",
            "meanFitheta",
            "meanAmplitudebeta",
            "powerbeta",
            "meanAbeta",
            "meanFibeta",
            "powergamma"
        ]

['kurtosis',
 'mean',
 'meanAmplitude',
 'meanFrequency',
 'median',
 'medianFrequency',
 'skewness',
 'totalPower',
 'variance',
 'meanAmplitudedelta',
 'powerdelta',
 'meanAdelta',
 'meanFidelta',
 'meanAmplitudealpha',
 'poweralpha',
 'meanAalpha',
 'meanFialpha',
 'meanAmplitudetheta',
 'powertheta',
 'meanAtheta',
 'meanFitheta',
 'meanAmplitudebeta',
 'powerbeta',
 'meanAbeta',
 'meanFibeta',
 'powergamma']

In [20]:
len(featureList)

20

In [21]:
labelEncoder_prev = LabelEncoder()
classes = labelEncoder_prev.fit_transform(classes)

c:\ProgramData\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
trainingInputs, testInputs, trainingClasses, testClasses = train_test_split(features, classes, test_size=0.25)

In [23]:
classifier = KerasClassifier(build_fn=createNetwork)
parameters = {
    'batch_size': [30, 35],
    'epochs': [1000, 2000],
    'inputShape': [(len(featureList),)],
    'hiddenLayerUnits': [[8], [12], [12, 12], [round(len(featureList)/2)]],
    'kernel_initializer': ['random_uniform', 'normal'],
    'optimizer': ['adam', 'adamax'],
    'loss': ['binary_crossentropy'],
    
}

C:\Users\Greze\AppData\Local\Temp\ipykernel_7296\2190092388.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn=createNetwork)


In [24]:
base

,kurtosis,mean,meanAmplitude,meanFrequency,median,medianFrequency,skewness,totalPower,variance,zzclassification,...,meanbeta,meanAmplitudebeta,powerbeta,meanAbeta,meanFibeta,meangamma,meanAmplitudegamma,powergamma,meanAgamma,meanFigamma
0,11.5723,-0.310300,0.64558,0.084093,-0.615560,0.031330,1.245500,2494.0973,60.8402,TA,...,-0.000459,1.00790,48.0378,1.3049,13.1943,-0.000324,0.69897,24.3555,1.08950,13.6041
1,7.0162,-0.314940,0.20093,0.051268,-0.827660,0.024136,1.058400,2783.4228,67.9031,TA,...,-0.000803,0.95488,30.2312,1.2011,12.4959,-0.000505,0.32039,5.5139,0.54754,8.4207
2,3.6025,-1.321800,1.23030,0.036277,-2.901600,0.018814,0.414100,21548.0296,525.0551,TA,...,0.005447,2.56130,211.7835,3.2146,33.2020,0.003836,0.72781,29.7440,1.14070,18.4952
4,3.9980,1.023900,5.20380,0.045542,-0.903780,0.023091,0.379710,28025.9247,683.6762,TA,...,-0.001976,2.91680,318.9589,3.7166,38.5210,-0.000680,0.79973,51.7185,1.25720,20.4667
5,7.7734,-2.064500,1.09900,0.045057,-2.650700,0.020930,0.116430,21138.1890,513.4533,TA,...,0.002950,2.59180,258.8602,3.3874,33.7424,0.001868,0.83125,34.7109,1.28830,21.0773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,3.2226,-0.063058,1.74540,0.045462,-0.007136,0.023847,0.049161,15315.6161,373.9762,TA,...,0.005440,1.66830,90.3816,2.1686,21.6588,0.003516,0.58207,19.0544,0.92788,15.0425
73,3.9027,-1.159700,1.05180,0.042820,-2.256000,0.018735,0.216970,28952.4461,706.1121,HVS,...,0.001153,2.16720,148.4985,2.7236,28.3230,0.000785,0.64721,18.0857,0.97812,16.0203
74,2.7070,-0.543080,2.31060,0.055341,-1.003900,0.018504,0.068108,12776.0178,311.7795,HVS,...,-0.001816,2.50190,188.0247,3.1893,32.8847,-0.001347,0.75866,24.1519,1.15050,18.9578
75,4.7007,-0.427010,2.99220,0.045443,0.125820,0.022391,0.176190,31920.7316,779.3291,TA,...,0.004711,2.41950,202.5840,3.0583,30.4531,0.002415,0.68570,21.0479,1.03300,16.9512


In [25]:
gridSearch = GridSearchCV(
    estimator=classifier,
    param_grid=parameters,
    scoring='accuracy',
    cv=5    
)

gridSearch = gridSearch.fit(
    trainingInputs,
    trainingClasses,
    verbose=0,
    use_multiprocessing = True
)

1/1 [==============================] - 0s 51ms/step


In [26]:
best_params = gridSearch.best_params_
best_accuracy = gridSearch.best_score_

In [27]:
logFile = 'modelLog.json'

execution_date = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
accuracy = best_accuracy
params = best_params
append_log(execution_date, accuracy, params, featureList, logFile, baseFile)

In [28]:
print(best_accuracy)
print(best_params)

0.7555555555555555
{'batch_size': 30, 'epochs': 2000, 'hiddenLayerUnits': [12], 'inputShape': (20,), 'kernel_initializer': 'normal', 'loss': 'binary_crossentropy', 'optimizer': 'adamax'}


In [4]:
l1 = [
            "kurtosis",
            "mean",
            "meanAmplitude",
            "meanFrequency",
            "median",
            "medianFrequency",
            "skewness",
            "totalPower",
            "variance",
            "meanAmplitudedelta",
            "powerdelta",
            "meanAdelta",
            "meanFidelta",
            "meanAmplitudealpha",
            "poweralpha",
            "meanAalpha",
            "meanFialpha",
            "meanAmplitudetheta",
            "powertheta",
            "meanAtheta",
            "meanFitheta",
            "meanAmplitudebeta",
            "powerbeta",
            "meanAbeta",
            "meanFibeta",
            "powergamma"
        ]


l2 = [
            "kurtosis",
            "mean",
            "meanAmplitude",
            "median",
            "skewness",
            "totalPower",
            "variance",
            "meandelta",
            "meanAmplitudedelta",
            "powerdelta",
            "meanAdelta",
            "meanFidelta",
            "meanalpha",
            "meanAmplitudealpha",
            "poweralpha",
            "meanAalpha",
            "meanFialpha",
            "meantheta",
            "meanAmplitudetheta",
            "powertheta",
            "meanAtheta",
            "meanFitheta",
            "meanbeta",
            "meanAmplitudebeta",
            "powerbeta",
            "meanAbeta",
            "meanFibeta",
            "powergamma"
        ]



difference = [item for item in l2 if item not in l1]
difference2 = [item for item in l1 if item not in l2]

In [5]:
difference
print(len(l1), len(l2))
print(difference)
print(difference2)

26 28
['meandelta', 'meanalpha', 'meantheta', 'meanbeta']
['meanFrequency', 'medianFrequency']
